# RGI-07: Region 12 (Caucasus and Middle East)

F. Maussion & S. Galos, June 2021

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
import os
from utils import mkdir

## Files and storage paths

In [2]:
# Region of interest 
reg = 12

# go down from rgi7_scripts/workflow
data_dir = '../../rgi7_data/'

# Level 2 GLIMS files
l2_dir = os.path.join(data_dir, 'l2_sel_reg_tars')

# Output directories
output_dir = mkdir(os.path.join(data_dir, 'l3_rgi7a'))
output_dir_tar = mkdir(os.path.join(data_dir, 'l3_rgi7a_tar'))

# RGI v6 file for comparison later 
rgi6_reg_file = os.path.join(data_dir, 'l0_RGIv6', '12_rgi60_CaucasusMiddleEast.zip')

In [3]:
# Specific to this region: boxes where data has to be selected differently
support_dir = os.path.join(data_dir, 'l0_support_data')
box_file = os.path.join(support_dir, 'RGI07_R12_UCA.tar.gz')

### Load the input data

In [4]:
# Read L2 files
shp = gpd.read_file('tar://' + l2_dir + f'/RGI{reg:02d}.tar.gz/RGI{reg:02d}/RGI{reg:02d}.shp')

### List of submissions 

In [5]:
toprint = []
for subid in shp.subm_id.unique():
    s_loc = shp.loc[shp.subm_id == subid]
    s = ''
    for c in ['subm_id', 'analysts', 'src_date']:
        toprint = s_loc[c].unique()
        if c != 'src_date':
            s += ' ' + (str(toprint[0]))
        else:
            for d in toprint:
                s += ' ' + d[:4]
    print(s)

 76.0 Nosenko, Gennadi (Gennady) 2004
 79.0 Nosenko, Gennadi (Gennady) 2004
 81.0 Nosenko, Gennadi (Gennady) 2004
 83.0 Nosenko, Gennadi (Gennady) 2004
 87.0 Nosenko, Gennadi (Gennady) 2004
 327.0 Khromova, Tatyana 2001
 341.0 Seliverstov, Yriy 2000
 423.0 Nosenko, Gennadi (Gennady) 2004
 424.0 Nosenko, Gennadi (Gennady) 1999
 425.0 Khromova, Tatyana 1999
 426.0 Nosenko, Gennadi (Gennady) 1999
 427.0 Nosenko, Gennadi (Gennady) 2001
 428.0 Nosenko, Gennadi (Gennady) 2004
 429.0 Nosenko, Gennadi (Gennady) 2004
 588.0 Ashkan, Farokhnia; Karimi, Neamat; Sara, Sheshangosht 2011
 642.0 Tielidze, Levan 2014 2013 2014 2014 2011 2016
 643.0 Tielidze, Levan 1987 1985 1986 1986 1986 1987
 644.0 Tielidze, Levan 1960
 662.0 Sarikaya, Mehmet Akif 2008 2006 2006 2004 2008 2009 2011
 724.0 Tielidze, Levan 1999 2000 2000 2000 1999 2002


### Apply selection criteria to create the RGI7 data subset

In [6]:
# try to get the data relevant for RGI07 and select by attributes - 
# for Caucasus we use 'subm_id' 642 by analyst 'Tielidze, Levan' which corresponds to the most recent (ca. 2013)
# of three available data sets for Caucasus by this anayst (others are from around 1960 and appr. 1985 respectively)
RGI_ss_caucasus = shp.loc[shp['subm_id'] == 724]
len(RGI_ss_caucasus)

2185

In [7]:
# try to get the data relevant for RGI07 and select by attributes - for Middle East: 'subm_id' 588 ()
RGI_ss_middle_east = shp.loc[shp['subm_id'] == 588]
len(RGI_ss_middle_east)

484

In [8]:
RGI_ss_middle_east.iloc[0]

line_type                                            glac_bound
anlys_id                                               297051.0
glac_id                                          G049337E33320N
anlys_time                                  2014-07-05T00:00:00
area                                                4651.577985
db_area                                                0.004652
width                                                       0.0
length                                                      0.0
primeclass                                                  0.0
min_elev                                                    0.0
mean_elev                                                   0.0
max_elev                                                    0.0
src_date                                    2011-09-01T00:00:00
rec_status                                                 okay
glac_name                                                  None
wgms_id                                 

In [9]:
# combine the two geodataframes
RGI_ss = RGI_ss_caucasus.append(RGI_ss_middle_east)

### Should we add some 1985 glaciers for completeness?

Hugonnet et al made a subjective choice to add some 1985 glaciers... should we as well?

In [10]:
# We decided not to for now
# EG:
# RGI_ss_caucasus_1985 = shp.loc[shp['subm_id'] == 644]
# len(RGI_ss_caucasus_1985)

### Should we add missing glaciers (only a few in peripheral areas) based on RGI06?

This is done based on a manually created shapefile mask containing the few glacierized areas not considered in the above data. Note that many of the glaciers added by this procedure are not glaciers by common definition but rock glaciers or even areas without ice in reality. Apart from that, outlines partly suffer from incorrect spatial positioning etc...  

In [11]:
# load a shapefile containing polygons which define areas uncovered by newly created RGI07 file
UCA = gpd.read_file('tar://' + box_file + '/RGI07_R12_UCA/RGI07_R12_UCA.shp')

In [12]:
# extract glacier outlines for regions mentioned above from glims data
df_UCA = gpd.overlay(shp, UCA, how='intersection')

In [13]:
# select submission ID of interesst (for RGI07 only subm_ID 662 available)
df_UCA = df_UCA.loc[df_UCA['subm_id']== 724]

In [14]:
# append df_UCA to RGI_ss
RGI_ss = RGI_ss.append(df_UCA, sort = False);

## Write out and tar 

In [15]:
dd = mkdir(f'{output_dir}/RGI{reg:02d}/', reset=True)

print('Writing...')
RGI_ss.to_file(dd + f'RGI{reg:02d}.shp')

print('Taring...')
print(subprocess.run(['tar', '-zcvf', f'{output_dir_tar}/RGI{reg:02d}.tar.gz', '-C', output_dir, f'RGI{reg:02d}']))

Writing...
Taring...
RGI12/
RGI12/RGI12.cpg
RGI12/RGI12.dbf
RGI12/RGI12.prj
RGI12/RGI12.shp
RGI12/RGI12.shx
CompletedProcess(args=['tar', '-zcvf', '../../rgi7_data/l3_rgi7a_tar/RGI12.tar.gz', '-C', '../../rgi7_data/l3_rgi7a', 'RGI12'], returncode=0)


## New RGI-file created - Check result!

### load reference data (here RGI6) to enable comparison

In [16]:
# Just to know the name of the file to open from zip
import zipfile
with zipfile.ZipFile(rgi6_reg_file, "r") as z:
    for f in z.filelist:
        if '.shp' in f.filename:
            fname = f.filename

# load reference data
ref_odf = gpd.read_file('zip://' + rgi6_reg_file + '/' + fname)

## Compare new RGI-file and reference data set 

### Number of elements (differences do not necessarily depict major problems)

In [17]:
print('Number of glaciers in new RGI subset:', len(RGI_ss))
print('Number of glaciers in reference data:', len(ref_odf))
print('Difference:', len(RGI_ss)-len(ref_odf))

Number of glaciers in new RGI subset: 2669
Number of glaciers in reference data: 1888
Difference: 781


### How many nominal glaciers were there in RGI06-R12?

In [54]:
len(ref_odf.loc[ref_odf.Status == 2])
ref_odf.Status?


Type:        Series
String form:
0       0
1       0
2       0
3       0
4       0
       ..
1883    0
1884    0
1885    0
1886    0
1887    0
Name: Status, Length: 1888, dtype: int64
Length:      1888
File:        ~/anaconda3/envs/geo_env/lib/python3.8/site-packages/pandas/core/series.py
Docstring:  
One-dimensional ndarray with axis labels (including time series).

Labels need not be unique but must be a hashable type. The object
supports both integer- and label-based indexing and provides a host of
methods for performing operations involving the index. Statistical
methods from ndarray have been overridden to automatically exclude
missing data (currently represented as NaN).

Operations between Series (+, -, /, *, **) align values based on their
associated index values-- they need not be the same length. The result
index will be the sorted union of the two indexes.

Parameters
----------
data : array-like, Iterable, dict, or scalar value
    Contains data stored in Series. If data is

### Total area

In [19]:
# add an area field to RGI_ss and reference data
RGI_ss['area'] = RGI_ss.to_crs({'proj':'cea'}).area
ref_odf['area'] = ref_odf.to_crs({'proj':'cea'}).area

In [20]:
# print and compare area values
Area_RGI = RGI_ss['area'].sum() * 1e-6
print('Area RGI [km²]:', Area_RGI)
Area_ref = ref_odf['area'].sum() * 1e-6
print('Area Ref [km²]:', Area_ref)
d = (Area_RGI - Area_ref)
print('Area difference [km²]:', d)

Area RGI [km²]: 1421.5735921911878
Area Ref [km²]: 1306.999710204219
Area difference [km²]: 114.5738819869689


### Here starts the comparison `RGI7a` with the original `Levan` dataset:
The data is downloaded beforehand from https://zenodo.org/record/5116329#.YXqvaL9Byit

Download: https://zenodo.org/record/5116329/files/CAUCASUS_GLACIERS_2000_2020_Tielidze%20et%20al.%202021.zip?download=1

In [52]:
# Read original data:
data_dir_lev = '../../rgi7_data/CAUCASUS_GLACIERS_2000_2020_Tielidze_et_al._2021'

lev_RU = gpd.read_file('../../rgi7_data/CAUCASUS_GLACIERS_2000_2020_Tielidze_et_al._2021/CAU_GLAC_2000/CAU_GLAC_RUS_2000.shp')
lev_AZ = gpd.read_file('../../rgi7_data/CAUCASUS_GLACIERS_2000_2020_Tielidze_et_al._2021/CAU_GLAC_2000/CAU_GLAC_AZER_2000.shp')
lev_GE = gpd.read_file('../../rgi7_data/CAUCASUS_GLACIERS_2000_2020_Tielidze_et_al._2021/CAU_GLAC_2000/CAU_GLAC_GEO_2000.shp')

lev = pd.concat([lev_RU, pd.concat([lev_AZ, lev_GE])])

804
24
1358


,OBJECTID,name,river_basi,main_river,SHAPE_Leng,SHAPE_Area,aspect,data,proc_desc,geogr_area,chief_affl,submitters,analysts,area,geometry,Name,MAIN_BASIN,sybmitters
0,25,None,50,Baksan,697.628503,2.818993e+04,N,2000-08-12 and 2000-09-05 and 2000-09-12,Using the GIS 10.6.1 software manual digitizin...,Russian Caucasus,"Victoria University of Wellington, New Zealand","Tielidze, Levan","Tielidze, Levan and Paul, Frank",0.028190,"POLYGON ((316853.444 4789815.030, 316847.094 4...",NaN,NaN,NaN
1,53,Tiutiurgy,Gara_Auzusu,Chegem,16675.003207,3.553812e+06,NW,2000-08-12 and 2000-09-05,Using the GIS 10.6.1 software manual digitizin...,Russian Caucasus,"Victoria University of Wellington, New Zealand","Tielidze, Levan","Tielidze, Levan and Paul, Frank",3.553810,"POLYGON ((343598.191 4775069.867, 343594.223 4...",NaN,NaN,NaN
2,90,None,Bashil_Auzusu,Chegem,1914.490508,1.122588e+05,NW,2000-08-12 and 2000-09-05 and 2000-09-12,Using the GIS 10.6.1 software manual digitizin...,Russian Caucasus,"Victoria University of Wellington, New Zealand","Tielidze, Levan","Tielidze, Levan and Paul, Frank",0.112259,"POLYGON ((333462.008 4782705.007, 333468.626 4...",NaN,NaN,NaN
3,127,None,Teberda,Kuban,457.716567,1.520325e+04,N,2000-09-12,Using the GIS 10.6.1 software manual digitizin...,Russian Caucasus,"Victoria University of Wellington, New Zealand","Tielidze, Levan","Tielidze, Levan and Paul, Frank",0.015203,"POLYGON ((243531.861 4789276.299, 243532.386 4...",NaN,NaN,NaN
4,150,None,Bashil_Auzusu,Chegem,8491.911677,2.061720e+06,N,2000-08-12 and 2000-09-05 and 2000-09-12,Using the GIS 10.6.1 software manual digitizin...,Russian Caucasus,"Victoria University of Wellington, New Zealand","Tielidze, Levan","Tielidze, Levan and Paul, Frank",2.061720,"POLYGON ((334422.462 4781229.461, 334407.594 4...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,1919,NaN,NaN,NaN,884.495175,4.724172e+04,NW,1999-09-01 and 2000-09-12 and 2001-08-05,Using the GIS 10.6.1 software manual digitizin...,Georgian Caucasus,"Victoria University of Wellington, New Zealand",NaN,"Tielidze, Levan and Paul, Frank",0.047242,"POLYGON ((247975.078 4783602.934, 247981.188 4...",None,Kodori,"Tielidze, Levan"
800,1920,NaN,NaN,NaN,1332.117325,1.025520e+05,NE,1999-09-01 and 2000-09-12 and 2001-08-05,Using the GIS 10.6.1 software manual digitizin...,Georgian Caucasus,"Victoria University of Wellington, New Zealand",NaN,"Tielidze, Levan and Paul, Frank",0.102552,"POLYGON ((245721.884 4791792.549, 245718.454 4...",50,Kodori,"Tielidze, Levan"
801,1921,NaN,NaN,NaN,740.059321,3.763786e+04,NE,1999-09-01 and 2000-09-12 and 2001-08-05,Using the GIS 10.6.1 software manual digitizin...,Georgian Caucasus,"Victoria University of Wellington, New Zealand",NaN,"Tielidze, Levan and Paul, Frank",0.037638,"POLYGON ((202613.762 4795762.861, 202605.295 4...",50,Kodori,"Tielidze, Levan"
802,1922,NaN,NaN,NaN,672.654745,2.705299e+04,NE,1999-09-01 and 2000-09-12 and 2001-08-05,Using the GIS 10.6.1 software manual digitizin...,Georgian Caucasus,"Victoria University of Wellington, New Zealand",NaN,"Tielidze, Levan and Paul, Frank",0.027053,"POLYGON ((204149.362 4795169.730, 204149.362 4...",50,Kodori,"Tielidze, Levan"


In [53]:
print('Number of glaciers in new RGI subset:', len(RGI_ss))
print('Number of glaciers in reference data:', len(lev))
print('Difference:', len(RGI_ss)-len(lev))

Number of glaciers in new RGI subset: 2669
Number of glaciers in reference data: 2186
Difference: 483


In [ ]:
# add an area field to RGI_ss and reference data
RGI_ss['area'] = RGI_ss.to_crs({'proj':'cea'}).area
lev['area'] = lev.to_crs({'proj':'cea'}).area

In [56]:
# print and compare area values
Area_RGI = RGI_ss['area'].sum() * 1e-6
print('Area RGI [km²]:', Area_RGI)
Area_lev = lev['area'].sum() # already in km² (?)
print('Area Ref (lev) [km²]:', Area_lev)
d = (Area_RGI - Area_lev)
print('Area difference [km²]:', d)

Area RGI [km²]: 1421.5735921911878
Area Ref (lev) [km²]: 1381.54838484
Area difference [km²]: 40.02520735118787
